# Rapid Refresh Model Plotting 

This notebook gives several examples of plotting model analyses from the RAP model.

In [4]:
#Required Libraries
from datetime import datetime, timedelta

from model_functions import *

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader

import matplotlib.pyplot as plt
import matplotlib.colors

from metpy.plots import StationPlot
from metpy.units import units
import metpy.calc as mpcalc

import numpy as np

from netCDF4 import Dataset, num2date

from scipy.ndimage import gaussian_filter

from xarray.backends import NetCDF4DataStore
import xarray as xr

import pandas as pd

In [5]:
#set time to plot (no leading zeros)
start_year  = 2021
start_month = 12
start_day   = 10
start_hour  = 12
start_min   = 0
    
#location to plot
#location = True
location_lat = 37.5
location_lon = -98.5
#    OR
#analysis bounds
location = False
lat_min = 30
lat_max = 45
lon_min = -110
lon_max = -90

In [6]:
dt = datetime(start_year,start_month,start_day,start_hour)
ncss = get_rap_dataset(dt)
#print(ncss)
#for i in ncss.variables:
#    if "isobaric" in i:
#        print(i)

https://www.ncei.noaa.gov/thredds/catalog/model-rap130anl/202112/20211210/catalog.xml


In [ ]:
query = ncss.query()
query.lonlat_box(north=lat_max,south=lat_min,east=lon_max,west=lon_min)
query.all_times()
query.add_lonlat()
query.accept('netcdf')
query.variables('Relative_humidity_isobaric',
                'u-component_of_wind_isobaric',
                'v-component_of_wind_isobaric',
                'Temperature_isobaric',
                'Geopotential_height_isobaric'
)
data = ncss.get_data(query)
ds = xr.open_dataset(NetCDF4DataStore(data)).metpy.parse_cf()

In [ ]:
dwpt = mpcalc.dewpoint_from_relative_humidity(ds.Temperature_isobaric, ds.Relative_humidity_isobaric)
spec_humidity = mpcalc.specific_humidity_from_dewpoint(ds.isobaric, dwpt)

print(ds.isobaric.shape)

In [ ]:
isent_levs = np.arange(290,321,2) * units.kelvin

#get isentropic data
isen_ds = mpcalc.isentropic_interpolation_as_dataset(
    isent_levs,
    ds.Temperature_isobaric[0],
    ds.Geopotential_height_isobaric[0],
    ds.Relative_humidity_isobaric[0],
    ds['u-component_of_wind_isobaric'][0],
    ds['v-component_of_wind_isobaric'][0]
)

#smooth parameters
isen_ds['pressure'] = mpcalc.smooth_n_point(isen_ds.pressure, 9, 10)
isen_ds['Relative_humidity_isobaric'] = mpcalc.smooth_n_point(isen_ds.Relative_humidity_isobaric, 9, 10)
isen_ds['temperature'] = mpcalc.smooth_n_point(isen_ds.temperature, 9, 10)
isen_ds['Geopotential_height_isobaric'] = mpcalc.smooth_n_point(isen_ds.Geopotential_height_isobaric, 9, 10)

In [ ]:
isen_level = 300*units.kelvin

proj = ccrs.LambertConformal(central_longitude=-95,central_latitude=35,standard_parallels=[35])



fig = plt.figure(figsize=(14,12))
ax = fig.add_subplot(1,1,1,projection=proj)

cs = ax.contour(ds.lon,ds.lat,isen_ds.pressure.metpy.sel(isentropic_level=isen_level),
                   np.arange(100,1100,50),colors='black',transform=ccrs.PlateCarree())
ax.clabel(cs,inline=True,fmt=lambda v: format(v,'.0f')[:3],zorder=10)


wind_slice = (slice(None,None,20),slice(None,None,20))
wb = ax.barbs(ds.lon[wind_slice].values,ds.lat[wind_slice].values,
            isen_ds["u-component_of_wind_isobaric"].metpy.sel(isentropic_level=isen_level)[wind_slice].metpy.convert_units('kts').values,
            isen_ds["v-component_of_wind_isobaric"].metpy.sel(isentropic_level=isen_level)[wind_slice].metpy.convert_units('kts').values,
            length=7,lw=1,pivot='middle',color='black',transform=ccrs.PlateCarree())

ax.add_feature(cfeature.STATES,edgecolor='k',linewidth=0.7)
ax.add_feature(cfeature.COASTLINE,edgecolor='k',linewidth=1.0)
ax.set_extent((lon_min,lon_max,lat_min,lat_max))

tulsa = [36.15, -95.99]
kc_mo = [39.10, -94.58]

ax.scatter(tulsa[1],tulsa[0],marker='*',color='k',s=150,transform=ccrs.PlateCarree())
ax.annotate("TULSA",(tulsa[1]-1.50,tulsa[0]-0.40),fontsize=15,transform=ccrs.PlateCarree())

ax.scatter(kc_mo[1],kc_mo[0],marker='*',color='k',s=150,transform=ccrs.PlateCarree())
ax.annotate("KANSAS CITY",(kc_mo[1]-3,kc_mo[0]-0.40),fontsize=15,transform=ccrs.PlateCarree())


plt.title('300-K RAP Isentropic Chart, Pressure (hPa),'
          ' and Wind Barbs (kt)', loc='left')
vtime = ds.time[0].data.astype('datetime64[ms]').astype('O')
plt.title('Valid Time: {}'.format(vtime), loc='right')
plt.savefig('isentropic_chart.png',bbox_inches='tight')